In [11]:
input_path ="C:\\Users\\arpit\\Downloads\\dyson\\input_files"

In [2]:
pwd

'c:\\Users\\arpit\\Downloads\\dyson\\code'

In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_timestamp

import yaml

# Create a Spark session
spark = SparkSession.builder.appName("ETL").getOrCreate()


In [55]:
# Function to read the configuration from config.yaml
def read_config():
    try:
        with open('config.yaml', 'r') as config_file:
            config = yaml.safe_load(config_file)
        return config
    except FileNotFoundError:
        print("Error: config.yaml file not found.")
        return None
    except yaml.YAMLError as e:
        print(f"Error loading config.yaml: {e}")
        return None

if __name__ == "__main__":
    # Read the configuration from config.yaml
    config = read_config()

    if config:
        # Access the input_path from the config
        input_path = config.get('input_path')
        output_path = config.get('output_path')

        # Print the input_path (you can use it in your PySpark pipeline)
        print(f"Input Path: {input_path}")
        print(f"Output Path: {output_path}")
    else:
        print("Configuration not loaded. Please check the YAML file and its location.")


Input Path: C:\Users\arpit\Downloads\dyson\source
Output Path: C:\Users\arpit\Downloads\dyson\target


In [73]:
# Read the CSV file into a DataFrame
df_char = spark.read.csv(f"{input_path}\characters.csv", header=True, inferSchema=True)
df_char.show(5)

+-----------+---------------+
|characterID|           name|
+-----------+---------------+
|    1009220|Captain America|
|    1010740| Winter Soldier|
|    1009471|      Nick Fury|
|    1009552|   S.H.I.E.L.D.|
|    1009228|  Sharon Carter|
+-----------+---------------+
only showing top 5 rows



In [72]:
# Read the CSV file into a DataFrame
df_stats = spark.read.csv(f"{input_path}\charcters_stats.csv", header=True, inferSchema=True)
df_stats = df_stats.withColumnRenamed("Name","name")
df_stats.show(5)

+-----------+---------+------------+--------+-----+----------+-----+------+-----+
|       name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat|Total|
+-----------+---------+------------+--------+-----+----------+-----+------+-----+
|    3-D Man|     good|          50|      31|   43|        32|   25|    52|  233|
|     A-Bomb|     good|          38|     100|   17|        80|   17|    64|  316|
| Abe Sapien|     good|          88|      14|   35|        42|   35|    85|  299|
|   Abin Sur|     good|          50|      90|   53|        64|   84|    65|  406|
|Abomination|      bad|          63|      80|   53|        90|   55|    95|  436|
+-----------+---------+------------+--------+-----+----------+-----+------+-----+
only showing top 5 rows



In [71]:
df_char_stats = df_char.join(df_stats, on="name", how="inner")
# Add audit columns
df_char_stats = df_char_stats.withColumn("batch_id", lit("101"))
df_char_stats = df_char_stats.withColumn("load_date", current_timestamp())



df_char_stats.show(5)


+---------------+-----------+---------+------------+--------+-----+----------+-----+------+-----+--------+--------------------+
|           name|characterID|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat|Total|batch_id|           load_date|
+---------------+-----------+---------+------------+--------+-----+----------+-----+------+-----+--------+--------------------+
|Captain America|    1009220|     good|          63|      19|   35|        56|   46|   100|  319|     101|2023-09-12 23:59:...|
| Winter Soldier|    1010740|     good|          56|      32|   35|        65|   60|    84|  332|     101|2023-09-12 23:59:...|
|      Nick Fury|    1009471|     good|          75|      11|   23|        42|   25|   100|  276|     101|2023-09-12 23:59:...|
|       Punisher|    1009515|     good|          50|      16|   23|        28|   22|   100|  239|     101|2023-09-12 23:59:...|
|      Red Skull|    1009535|      bad|          75|      10|   12|        14|   19|    80|  210|     10

C:\Users\arpit\Downloads\dyson\target


In [88]:
# Write the DataFrame to Parquet format
try:
    df_char_stats.write.parquet(f"{output_path}/characters_stats_test1.parquet", mode="overwrite")
except Exception as e:
    print(f"Error writing DataFrame to Parquet: {str(e)}")


Error writing DataFrame to Parquet: An error occurred while calling o341.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocal